# NOTE: data is updated every day/week

### Set up cron job to download and extract data daily

In [ ]:
https://download.geofabrik.de/north-america/us.html


In [ ]:
# OSRM
# snap GPS coords to road

# e.g. DC
http://download.geofabrik.de/north-america/us/district-of-columbia-latest.osm.pbf


In [ ]:
docker run -t -v "${PWD}:/data" osrm/osrm-backend osrm-extract -p /opt/car.lua /data/district-of-columbia-latest.osm.pbf

# NOTE FILE EXTENSION
docker run -t -v "${PWD}:/data" osrm/osrm-backend osrm-partition /data/district-of-columbia-latest.osrm
docker run -t -v "${PWD}:/data" osrm/osrm-backend osrm-customize /data/district-of-columbia-latest.osrm

# Start routing engine HTTP server on port
# publish the container port (5000) to the host port (7000) so the container can communicate with  the “outside world”. NOTE: remember to allow HTTP connection (add Firewall rule) at tcp:7000
docker run -d --rm -p 7000:5000 --name osrm -v "/home/nhat/allocation/building-to-road:/data" osrm/osrm-backend osrm-routed --algorithm mld /data/district-of-columbia-latest.osrm
# docker run -d --rm -p 7000:5000 --name osrm -v "${PWD}:/data" osrm/osrm-backend osrm-routed --algorithm mld /data/district-of-columbia-latest.osrm
# docker run -d --rm -p 7001:5000 --name osrm1 -v "/home/nhat/allocation/building-to-road:/data" osrm/osrm-backend osrm-routed --algorithm mld /data/district-of-columbia-latest.osrm
# docker run -t -i -p 7000:5000 -v "${PWD}:/data" osrm/osrm-backend osrm-routed --algorithm mld /data/district-of-columbia-latest.osrm



In [ ]:
# coordinates {long}{lat}
-77.05019790313297,38.889425332914655 # lincoln memorial
-77.03660359320139,38.89709174367345 # white house

In [ ]:
# Make requests against the HTTP server
curl "http://35.223.14.85:7000/route/v1/driving/-77.05019790313297,38.889425332914655;-77.03660359320139,38.89709174367345?overview=false"
curl "http://35.223.14.85:7000/route/v1/driving/-77.05019790313297,38.889425332914655;-77.03660359320139,38.89709174367345?steps=true"


In [43]:
import requests
# GCE's OSRM backend is accepting requests at VM external IP, port 7000
# NEAREST request
url = "http://35.223.14.85:7000/nearest/v1/driving/-77.05019790313297,38.889425332914655"
r = requests.get(url)
res = r.json()
if res['code'] == 'Ok':
    nearest_loc, dis = res['waypoints'][0]['location'], res['waypoints'][0]['distance']
    # DISTANCES are in METER. Confirmed, using https://www.movable-type.co.uk/scripts/latlong.html
    print(nearest_loc, dis)

[-77.050262, 38.890174] 83.332448


In [13]:

# import polyline
# polyline.decode('qvjlFb{guMB{CyIqGm@I?st@wq@E@gHrQ@')